<a href="https://colab.research.google.com/github/FMurray/hyperdemocracy/blob/main/hyper_democracy_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
# if you are on a google colab, uncomment the lines below to fetch the requirements file and the hyperdemocracy.py module
# and pip install the requirements

#!wget https://raw.githubusercontent.com/FMurray/hyperdemocracy/main/requirements.txt
#!wget https://raw.githubusercontent.com/FMurray/hyperdemocracy/main/hyperdemocracy.py
!pip install -r requirements.txt
!pip install torch transformers

In [2]:
import os
import rich
import tiktoken
from transformers import pipeline
from rich import print
from hyperdemocracy import load_assembleco_records

In [3]:
# this is for development
%load_ext autoreload
%autoreload 2

In [4]:
records = load_assembleco_records(process=True, strip_html=True, remove_empty_body=True)

Found cached dataset parquet (/home/calliope/.cache/huggingface/datasets/assembleco___parquet/assembleco--hyperdemocracy-a598a9b2b17e51dc/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
records.head()

,key,name,sponsors,summary,body,themes,index,actions,amendments,committees,relatedbills,cosponsors,subjects,text,titles,congress_num,legis_class,legis_num,congress_gov_url,assembled_url
0,118HCONRES1,Regarding consent to assemble outside the seat...,"[[C001053, Rep. Cole, Tom [R-OK-4], sponsor]]",This concurrent resolution authorizes the Spe...,[Congressional Bills 118th Congress]\n[From th...,"[Congress, Congressional operations and organi...","{'bill': {'actions': {'count': 7, 'url': 'http...","{'actions': [{'actionCode': None, 'actionDate'...","{'amendments': [], 'pagination': {'count': 0},...","{'committees': [], 'request': {'billNumber': '...","{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [], 'pagination': {'count': 0, ...","{'pagination': {'count': 2}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,1,https://www.congress.gov/bill/118th-congress/h...,https://assembled.app/measures/118HCONRES1
1,118HCONRES10,Expressing the sense of Congress that the Unit...,"[[T000165, Rep. Tiffany, Thomas P. [R-WI-7], s...",This concurrent resolution calls on the Presi...,[Congressional Bills 118th Congress]\n[From th...,[International Affairs],"{'bill': {'actions': {'count': 4, 'url': 'http...","{'actions': [{'actionCode': 'H11100', 'actionD...","{'amendments': [], 'pagination': {'count': 0},...",{'committees': [{'activities': [{'date': '2023...,"{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [{'bioguideId': 'P000605', 'dis...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,10,https://www.congress.gov/bill/118th-congress/h...,https://assembled.app/measures/118HCONRES10
2,118HCONRES11,Providing for a joint session of Congress to r...,"[[S001176, Rep. Scalise, Steve [R-LA-1], spons...",This concurrent resolution provides for a joi...,[Congressional Bills 118th Congress]\n[From th...,"[Congress, Congressional operations and organi...","{'bill': {'actions': {'count': 10, 'url': 'htt...","{'actions': [{'actionCode': None, 'actionDate'...","{'amendments': [], 'pagination': {'count': 0},...","{'committees': [], 'request': {'billNumber': '...","{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [], 'pagination': {'count': 0, ...","{'pagination': {'count': 3}, 'request': {'bill...","{'pagination': {'count': 3}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,11,https://www.congress.gov/bill/118th-congress/h...,https://assembled.app/measures/118HCONRES11
3,118HCONRES12,Expressing the sense of Congress that all dire...,"[[C001039, Rep. Cammack, Kat [R-FL-3], sponsor...",This concurrent resolution expresses the sens...,[Congressional Bills 118th Congress]\n[From th...,"[Foreign Trade and International Finance, Agri...","{'bill': {'actions': {'count': 5, 'url': 'http...","{'actions': [{'actionCode': 'H11000', 'actionD...","{'amendments': [], 'pagination': {'count': 0},...",{'committees': [{'activities': [{'date': '2023...,"{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [{'bioguideId': 'K000380', 'dis...","{'pagination': {'count': 6}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,12,https://www.congress.gov/bill/118th-congress/h...,https://assembled.app/measures/118HCONRES12
4,118HCONRES13,Supporting the Local Radio Freedom Act.,"[[W000809, Rep. Womack, Steve [R-AR-3], sponso...",This concurrent resolution declares that Cong...,[Congressional Bills 118th Congress]\n[From th...,"[Science, Technology, Communications, Congress]","{'bill': {'actions': {'count': 3, 'url': 'http...","{'actions': [{'actionCode': 'H11100', 'actionD...","{'amendments': [], 'pagination': {'count': 0},...",{'committees': [{'activities': [{'date': '2023...,"{'paginati

In [6]:
records.shape

(6132, 20)

# From Pandas Dataframe to LangChain Documents

A langchain document is a simple class with two attributes, 
* page_content (a string)
* metadata (a dictionary)

In [7]:
from langchain.schema import Document 

In [9]:
all_docs = []
for i, record in records.iterrows():
    doc = Document(
        page_content=record['body'],
        metadata={
            # Note: chroma can only filter on float, str, or int
            # https://docs.trychroma.com/usage-guide#using-where-filters
            'key': record['key'],
            'congress_num': record['congress_num'],
            'legis_class': record['legis_class'],
            'legis_num': record['legis_num'],
            'name': record['name'],
            'summary': record['summary'],
            'sponsor': record['sponsors'][0][0],
            'source': record['congress_gov_url'] + "\n" + record['name'],
        },
    )
    all_docs.append(doc)
docs = all_docs[:50]

In [10]:
print(docs[0])

Document(
    page_content='[Congressional Bills 118th Congress]\n[From the U.S. Government Publishing Office]\n[H. Con. Res.
1 Engrossed in House (EH)]\n\n<DOC>\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  
\n  \n  \n  \n118th CONGRESS\n  1st Session\nH. CON. RES. 
1\n\n_______________________________________________________________________\n\n                         CONCURRENT
RESOLUTION\n\n    Resolved by the House of Representatives (the Senate concurring), \nThat pursuant to clause 4, 
section 5, article I of the Constitution, \nduring the One Hundred Eighteenth Congress the Speaker of the House and
\nthe Majority Leader of the Senate or their respective designees, acting \njointly after consultation with the 
Minority Leader of the House and \nthe Minority Leader of the Senate,\n\n\n              \n\n may notify the 
Members of the House and the Senate, respectively, to \nassemble at a place outside the District of Columbia if, in
their \nopinion, the public interest shall warrant it.\n\n            Passed the House of Representatives January 
9, 2023.\n\n            Attest:\n\n                                                                 Clerk.\n118th 
CONGRESS\n\n  1st Session\n\n                             H. CON. RES. 
1\n\n_______________________________________________________________________\n\n                         CONCURRENT
RESOLUTION\n\n     Regarding consent to assemble outside the seat of government.\n\n',
    metadata={
        'key': '118HCONRES1',
        'congress_num': '118',
        'legis_class': 'HCONRES',
        'legis_num': '1',
        'name': 'Regarding consent to assemble outside the seat of government.',
        'summary': ' This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the 
Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants 
it.',
        'sponsor': 'C001053',
        'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/1\nRegarding consent to
assemble outside the seat of government.'
    }
)

In [12]:
#from langchain import PromptTemplate, LLMChain

template = """Describe a procedure easing scenarios of {issue}.

Begin by: """

# prompt = PromptTemplate(template=template, input_variables=["issue"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)
# issue = "commercial relations in Asia, Europe, and Americas"

# print(llm_chain.run(issue))

# Remember: LangChain can slice up your corpus in "chunks".

-- https://python.langchain.com/docs/modules/data_connection/document_transformers/#text-splitters

Here are some useful options for splitting legislative text, 

* [character text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter)
  * How the text is split: by single character
  * How the chunk size is measured: by number of characters
* [recursive text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)
  * How the text is split: by list of characters
  * How the chunk size is measured: by number of characters
* [split by token](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/split_by_token)
  * How the text is split: by character passed in
  * How the chunk size is measured: by tiktoken tokenizer

If you are not familiar with the concept of a token, this article may help, 
* https://simonwillison.net/2023/Jun/8/gpt-tokenizers/

Mini Side Quest
* see if there is anything interesting that can be done with this https://twitter.com/RLanceMartin/status/1670489431168659456?s=20

In [13]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter

In [14]:
text = """We hold these truths to be self-evident, that all men are created equal,

that they are endowed by their Creator with certain unalienable Rights,

that among these are Life, Liberty and the pursuit of Happiness."""

## CharacterTextSplitter

In [ ]:
# this is the default separator
CharacterTextSplitter(separator="\n\n", chunk_size=20, chunk_overlap=0).split_text(text)

In [ ]:
# this is what happens if we change the default separator
CharacterTextSplitter(separator=" ", chunk_size=20, chunk_overlap=0).split_text(text)

In [ ]:
# this is what overlap does
CharacterTextSplitter(separator=" ", chunk_size=20, chunk_overlap=10).split_text(text)

## RecursiveCharacterTextSplitter

In [15]:
# these are the default separators
RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""], chunk_size=40, chunk_overlap=0).split_text(text)

['We hold these truths to be self-evident,',
 'that all men are created equal,',
 'that they are endowed by their Creator',
 'with certain unalienable Rights,',
 'that among these are Life, Liberty and',
 'the pursuit of Happiness.']

In [ ]:
# this is what happens if we add "," to the separators
RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ",", " ", ""], chunk_size=40, chunk_overlap=0).split_text(text)

## TokenTextSplitter

In [ ]:
# the length unit for chunk_size is now tokens not characters
TokenTextSplitter(model_name="text-embedding-ada-002", chunk_size=10, chunk_overlap=0).split_text(text)

In [ ]:
# same for chunk_overlap
TokenTextSplitter(model_name="text-embedding-ada-002", chunk_size=10, chunk_overlap=4).split_text(text)

In [ ]:
# note that "self-evident" is 4 tokens
enc = tiktoken.encoding_for_model("text-embedding-ada-002")
print(enc)
token_ids = enc.encode("self-evident")
token_strs = [enc.decode_single_token_bytes(token) for token in token_ids]
print("token_ids: ", token_ids)
print("token_strs: ", token_strs)

## Lets Make a TextSplitter Choice here

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_docs = text_splitter.split_documents(docs)

In [17]:
print("Number of original docs: ", len(docs))
print("Number of split docs: ", len(split_docs))

Number of original docs:  50

Number of split docs:  752

# Part 2 - Embed and Index Doc Chunks

Now we will embed and index the document chunks from the previous section. 
We have many choices when it comes to text embedding models and vector indexes. 
For this tutorial we will choose, 

* text embedding model: `text_embedding_ada_002`
* vector index:
  * https://www.trychroma.com
  * https://docs.trychroma.com/usage-guide#changing-the-distance-function
  * https://github.com/nmslib/hnswlib/tree/master#supported-distances
  * https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/chroma.py
  * https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/utils.py#L10

For a look at some of the top performing closed and open source text embedding models, check out the HuggingFace Massive Text Embedding Benchmark (MTEB), 
* https://huggingface.co/spaces/mteb/leaderboard
  
For a more detailed introduction to embeddings in general, see the embeddings notebook
* https://github.com/FMurray/hyperdemocracy/blob/main/sidequests/embeddings_v2.ipynb

In [22]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings;

embeddings = HuggingFaceEmbeddings
persist_directory = "hyperdemocracy-chromadb-local"
# model_name_or_path = os.environ.get("bigscience/bloom-3b", "all-mpnet-base-v2")
# encoder = SentenceTransformer(model_name_or_path='bigscience/bloom-3b')

In [23]:
vec_store = Chroma.from_documents(split_docs, embeddings, persist_directory=persist_directory)

TypeError: HuggingFaceEmbeddings.embed_documents() missing 1 required positional argument: 'texts'

In [ ]:
vec_store

## Save vectorstore to disk

In [ ]:
vec_store.persist()

## Reload vectorstore from disk

In [ ]:
vec_store = None
vec_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
vec_store

In [ ]:
ret_docs = vec_store.similarity_search_with_score(
    "nuclear power", 
    k=5, 
    filter={"source": "https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17"},
)

print("number of returned docs: ", len(ret_docs))
for doc in ret_docs:
    print(doc)

# Part 3 - Build A RetrievalQA Chain

In [ ]:
import langchain
langchain.verbose = False

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# base classes to examine
from langchain.vectorstores.base import VectorStore
from langchain.schema import BaseRetriever

## Create a Retriever from Chroma VectorStore

In [ ]:
retriever = vec_store.as_retriever(search_kwargs={'k':10})

In [ ]:
vec_store

In [ ]:
retriever

## Choose an LLM

With LangChain we can use a text completion model or a chat model for QA.

In [ ]:
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-3b",
    task="text-generation",
    model_kwargs={ "temperature": 0, "max_length": 64 }
)

In [ ]:
llm

In [ ]:
# create a RetrievalQA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [ ]:
answer = qa("What is the solution to climate change?")

In [ ]:
answer.keys()

In [ ]:
print(answer['query'])

In [ ]:
print(answer['result'])

In [ ]:
# because we set `return_source_documents=True` and k=10
# we can see the 10 document chunks that were returned
print(answer['source_documents'])

## Activity

* what are the components of the RetrievalQA chain?
* what is the QA prompt?
* how would you modify the QA prompt?
* what is the difference between the following qa chain types?,
    * stuff
    * map_reduce
    * map_rerank
    * refine
 
## Resources

* https://python.langchain.com/docs/modules/chains/document/

In [ ]:
# print(qa)

In [ ]:
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain

In [ ]:
BaseCombineDocumentsChain?

## CombineDocumentChains

* https://python.langchain.com/docs/modules/chains/document/stuff
* https://python.langchain.com/docs/modules/chains/document/refine
* https://python.langchain.com/docs/modules/chains/document/map_reduce
* https://python.langchain.com/docs/modules/chains/document/map_rerank

## Examine the RetrievalQA Prompt 

Note that the prompt template used will depend on the choice of LLM (text completion vs chat). 

In [ ]:
prompt_template = qa.combine_documents_chain.llm_chain.prompt
print(prompt_template)

# Part 4 - Create a RetrievalQAWithSourcesChain

Now we will do the same thing using a chain that provides sources in the generated answer.

In [ ]:
qaws = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [ ]:
answer = qaws("What is the solution to climate change?")

In [ ]:
answer.keys()

In [ ]:
print(answer['question'])

In [ ]:
print(answer['answer'])

In [ ]:
print(answer['sources'])

In [ ]:
print(answer['source_documents'])

## Activity 

* In this example, all of the returned document chunks came from one original document (118HCONRES37). What can be done to encourage a more diverse set of documents?
* What prompt is used? 

In [ ]:
pt = qaws.combine_documents_chain.llm_chain.prompt

In [ ]:
print(pt.format(summaries='[SUMMARIES]', question='[QUESTION]'))